<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/DIA_stockpredictrion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


In [3]:
# Define the tickers and their weights in DIA
tickers_weights = {
    'MSFT': 5.49, 'V': 4.92, 'HD': 4.62, 'CAT': 4.23, 'AAPL': 3.97, 'BA': 3.84, 'CRM': 3.46,
    'AXP': 3.39, 'AMZN': 3.30, 'GS': 3.16, 'HON': 3.13, 'NKE': 3.08, 'JPM': 2.92, 'MMM': 2.89,
    'INTC': 2.89, 'DIS': 2.89, 'TRV': 2.88, 'DOW': 2.87, 'MCD': 2.86, 'IBM': 2.83, 'WMT': 2.79,
    'CSCO': 2.76, 'VZ': 2.69, 'KO': 2.66, 'CVX': 2.64, 'PG': 2.64, 'MRK': 2.59, 'AMGN': 2.51,
    'JNJ': 2.50, 'UNH': 2.48
}

tickers = list(tickers_weights.keys()) + ['DIA']

In [26]:
# Fetch historical data
data = yf.download(tickers, start="2020-01-01", end="2024-07-24", group_by='ticker')  # Adjust end date as needed

[*********************100%%**********************]  31 of 31 completed


In [27]:
# Preprocess and create features
def preprocess_data(df):
    df['Previous Close'] = df['Close'].shift(1)
    df['Open-Close'] = df['Open'] - df['Previous Close']
    df['High-Low'] = df['High'] - df['Low']
    df = df.dropna()
    return df


In [28]:
# Process data for each ticker
processed_data = {ticker: preprocess_data(data[ticker]) for ticker in tickers}

In [29]:
# Aggregate features based on weights
def aggregate_features(processed_data, tickers_weights):
    aggregated_features = pd.DataFrame()
    for ticker, weight in tickers_weights.items():
        weighted_data = processed_data[ticker] * (weight / 100)
        if aggregated_features.empty:
            aggregated_features = weighted_data
        else:
            aggregated_features += weighted_data
    return aggregated_features

In [30]:
# Create aggregated features for the model
aggregated_features = aggregate_features(processed_data, tickers_weights)

# Use DIA data as the target
dia_data = processed_data['DIA']
y = dia_data['Open'].loc[aggregated_features.index]

In [31]:
processed_data

{'MSFT': Price             Open        High         Low       Close   Adj Close  \
 Date                                                                     
 2020-01-03  158.320007  159.949997  158.059998  158.619995  152.295425   
 2020-01-06  157.080002  159.100006  156.509995  159.029999  152.689072   
 2020-01-07  159.320007  159.669998  157.320007  157.580002  151.296906   
 2020-01-08  158.929993  160.800003  157.949997  160.089996  153.706802   
 2020-01-09  161.839996  162.220001  161.029999  162.089996  155.627014   
 ...                ...         ...         ...         ...         ...   
 2024-07-17  442.589996  444.850006  439.179993  443.519989  443.519989   
 2024-07-18  444.339996  444.649994  434.399994  440.369995  440.369995   
 2024-07-19  433.100006  441.140015  432.000000  437.109985  437.109985   
 2024-07-22  441.790009  444.600006  438.910004  442.940002  442.940002   
 2024-07-23  443.899994  448.390015  443.100006  444.850006  444.850006   
 
 Price         

In [32]:
# Ensure the features and target are aligned
X = aggregated_features.loc[y.index]

In [33]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [34]:
# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
# prompt: apply different ml mdoels

# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
lr_predictions = lr_model.predict(X_test)

# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Support Vector Regression
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train, y_train)
svr_predictions = svr_model.predict(X_test)

# LSTM (Long Short-Term Memory) - Requires reshaping data
X_train_lstm = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_lstm = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(1, X_train.shape[1])))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, verbose=0)
lstm_predictions = lstm_model.predict(X_test_lstm).flatten()

# Evaluate Models
def evaluate_model(predictions, model_name):
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    print(f"{model_name} - MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, R-squared: {r2:.4f}")

evaluate_model(lr_predictions, "Linear Regression")
evaluate_model(rf_predictions, "Random Forest")
evaluate_model(svr_predictions, "Support Vector Regression")
evaluate_model(lstm_predictions, "LSTM")


In [37]:
# Support Vector Regression
svr_model = SVR(kernel='rbf')
svr_model.fit(X_train, y_train)

SVR()

In [40]:
# Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [41]:
# Predict the next day's opening price
latest_data = X_test.iloc[-1].values.reshape(1, -1)
next_day_opening_price = model.predict(latest_data)[0]


In [42]:
# Print the predicted opening price for the next day
print(f"Predicted opening price for the next day: {next_day_opening_price}")

Predicted opening price for the next day: 412.34825661410355
